In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32,32,3)
num_classes = 10

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv('/content/gdrive/My Drive/fastcampus/cifar/train_dataset.csv')
train_df.head()

,name,class
0,/content/gdrive/My Drive/fastcampus/cifar/trai...,dog
1,/content/gdrive/My Drive/fastcampus/cifar/trai...,cat
2,/content/gdrive/My Drive/fastcampus/cifar/trai...,frog
3,/content/gdrive/My Drive/fastcampus/cifar/trai...,dog
4,/content/gdrive/My Drive/fastcampus/cifar/trai...,truck


In [6]:
test_df = pd.read_csv('/content/gdrive/My Drive/fastcampus/cifar/test_dataset.csv')
test_df.head()

,name,class
0,/content/gdrive/My Drive/fastcampus/cifar/test...,airplane
1,/content/gdrive/My Drive/fastcampus/cifar/test...,bird
2,/content/gdrive/My Drive/fastcampus/cifar/test...,ship
3,/content/gdrive/My Drive/fastcampus/cifar/test...,cat
4,/content/gdrive/My Drive/fastcampus/cifar/test...,horse


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.3,
    zoom_range = 0.2,
    horizontal_flip= True
)
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    
)

In [8]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'name',#filename,path가 있는 col
    y_col = 'class',#class, traget data
    target_size = input_shape[:2],#28x28
    batch_size = batch_size#batch사이즈만큼 생성해서 iterator를 내보냄
)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'name',#filename,path가 있는 col
    y_col = 'class',#class, traget data
    target_size = input_shape[:2],
    batch_size = batch_size 
)


KeyboardInterrupt: ignored

In [ ]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),#한 세대마다 데이터를 얼마나 얻을거냐.
    epochs = num_epochs,
    validation_data = test_generator,
    validation_steps = len(test_generator)
)